### Getting Reddit comments data

In [9]:
import nltk
import matplotlib
import re

from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

from bs4 import BeautifulSoup
import requests

import numpy as np

import praw
from praw.models import MoreComments

In [10]:
sub = "PrequelMemes"
ci = "wNnSYWwhkIvYFOgtGkt7eg" #your client id
cs = "Jm3VihJQnFsivAevI3iAR0iT8xzhWQ" #your client secret
ua = "anelka39" #your user agent name
reddit = praw.Reddit(
    client_id=ci,
    client_secret=cs,
    user_agent=ua
)

with open(sub+".txt", "w", encoding='utf-8') as f:


    #on the following line you can change top to any of the previously mentioned ways of sorting 
    #and the limit to however many posts you would like to extract (here we extract just 10).
    for post in reddit.subreddit(sub).top(limit=10): 

        #this line collects the post titles
        f.write(post.title+"\n")

        #this line collects the post content
        f.write(post.selftext+"\n")

        #this section collects the comments
        for comment in post.comments:
            if isinstance(comment, MoreComments):
                continue
            f.write(comment.body+"\n")
    print(f"completed downloading text from r/{sub}")

completed downloading text from r/PrequelMemes


In [11]:
def load_txt(file):
    with open(file, encoding='utf8') as f:
        return ' '.join(line.strip() for line in f.readlines())
from nltk.corpus import stopwords


In [12]:
%%time
sub = "PrequelMemes"
subr_txt = load_txt(f'{sub}.txt')
subr_txt = subr_txt.lower()

subr_tokens = nltk.word_tokenize(subr_txt)



subr_lowercase = [t.lower() for t in subr_tokens]
subr_no_punct = [t for t in subr_lowercase if t.isalnum()]
subr_no_stop = [t for t in subr_no_punct if t not in stopwords.words('english')]



CPU times: total: 4.89 s
Wall time: 5.23 s


In [13]:
print(subr_txt[:2000])

every day general grievous adds a unique lightsaber to his collection. day 66(finale): every u/raymesiris suggestion.  u/thibson34 has reddit premium for life from these posts so were all getting this on a t shirt right? i know one of you quarenteend at home has a screen press. let's get this started my lightsabers have doubled since the last time we met, r/prequelmemes u/thibson34: *stops the series*  general grievous: goodbye old friend, may the force be with you and thus it was, the fourth age of r/prequelmemes began. and the fellowship of grievous, though eternally bound by friendship and love, was ended.  nine and a half weeks to the day, since thibson sent us on our long journey, we found ourselves looking upon a familiar sight. we were home.  how do you pick up the threads of an old life? how do you go on, when in your heart, you begin to understand; there is no going back.  there are some things that time cannot mend. some hurts that go too deep... that have taken hold.  grievo

### Model Work

In [14]:
import gensim
nltk.download('word2vec_sample')

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


True

In [15]:
from nltk.data import find
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [16]:
len(model.index_to_key)

43981

In [18]:
fdist = nltk.FreqDist(subr_no_stop)
top_40 = fdist.most_common(40)

In [19]:
top_40

[('one', 139),
 ('day', 134),
 ('like', 113),
 ('lightsaber', 107),
 ('post', 105),
 ('time', 102),
 ('good', 98),
 ('love', 91),
 ('would', 80),
 ('great', 73),
 ('oh', 69),
 ('amazing', 68),
 ('upvotes', 67),
 ('could', 66),
 ('beautiful', 64),
 ('well', 64),
 ('reddit', 63),
 ('really', 63),
 ('meme', 62),
 ('ever', 60),
 ('top', 59),
 ('grant', 58),
 ('4h', 58),
 ('grievous', 57),
 ('sub', 55),
 ('comments', 55),
 ('3h', 55),
 ('may', 53),
 ('force', 53),
 ('made', 53),
 ('many', 52),
 ('make', 52),
 ('get', 51),
 ('man', 50),
 ('1', 49),
 ('fuck', 49),
 ('shit', 48),
 ('see', 48),
 ('thought', 46),
 ('seen', 45)]

In [20]:
tenwords=[]
for i in top_40:
    tenwords.append(i[0])
tenwords=tenwords[:10]
print(tenwords)

['one', 'day', 'like', 'lightsaber', 'post', 'time', 'good', 'love', 'would', 'great']


In [22]:
#manually replacing words not in model ('lightsaber' swapped for 'well')
# added in "meme" 
tenwords=['one', 'day', 'like', 'post', 'time', 'good', 'love', 'would', "well", "meme"]

In [24]:
for i in range(len(tenwords)):
    print(f"{i+1}, {tenwords[i]}:   {model.most_similar(positive=[tenwords[i]], topn = 5)}")
    

1, one:   [('only', 0.5982926487922668), ('two', 0.5769618153572083), ('three', 0.5686047077178955), ('five', 0.5628833174705505), ('four', 0.5627162456512451)]
2, day:   [('week', 0.6552987098693848), ('days', 0.6244534850120544), ('morning', 0.606966495513916), ('month', 0.6004591584205627), ('hours', 0.5727264285087585)]
3, like:   [('really', 0.5752449035644531), ('weird', 0.5676318407058716), ('crazy', 0.5382447838783264), ('kind', 0.5310240387916565), ('maybe', 0.5220044255256653)]
4, post:   [('posts', 0.5952650904655457), ('logged', 0.3730790317058563), ('posted', 0.35915204882621765), ('office', 0.34675872325897217), ('following', 0.3380518853664398)]
5, time:   [('day', 0.512919008731842), ('moment', 0.5121495723724365), ('days', 0.4926118552684784), ('period', 0.47037914395332336), ('periods', 0.45989590883255005)]
6, good:   [('great', 0.7291511297225952), ('bad', 0.7190051674842834), ('terrific', 0.6889115571975708), ('decent', 0.6837347149848938), ('nice', 0.6836092472076

### Part 2

In [25]:
import string
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

In [26]:
mydata = subr_txt
mydata_tokens = nltk.sent_tokenize(mydata)
mydata_tokens

['every day general grievous adds a unique lightsaber to his collection.',
 'day 66(finale): every u/raymesiris suggestion.',
 'u/thibson34 has reddit premium for life from these posts so were all getting this on a t shirt right?',
 'i know one of you quarenteend at home has a screen press.',
 "let's get this started my lightsabers have doubled since the last time we met, r/prequelmemes u/thibson34: *stops the series*  general grievous: goodbye old friend, may the force be with you and thus it was, the fourth age of r/prequelmemes began.",
 'and the fellowship of grievous, though eternally bound by friendship and love, was ended.',
 'nine and a half weeks to the day, since thibson sent us on our long journey, we found ourselves looking upon a familiar sight.',
 'we were home.',
 'how do you pick up the threads of an old life?',
 'how do you go on, when in your heart, you begin to understand; there is no going back.',
 'there are some things that time cannot mend.',
 'some hurts that go

In [27]:
mydata_tokens_no_punct = [''.join(letter for letter in word if letter not in string.punctuation) for word in mydata_tokens]
mydata_tokens_no_punct

['every day general grievous adds a unique lightsaber to his collection',
 'day 66finale every uraymesiris suggestion',
 'uthibson34 has reddit premium for life from these posts so were all getting this on a t shirt right',
 'i know one of you quarenteend at home has a screen press',
 'lets get this started my lightsabers have doubled since the last time we met rprequelmemes uthibson34 stops the series  general grievous goodbye old friend may the force be with you and thus it was the fourth age of rprequelmemes began',
 'and the fellowship of grievous though eternally bound by friendship and love was ended',
 'nine and a half weeks to the day since thibson sent us on our long journey we found ourselves looking upon a familiar sight',
 'we were home',
 'how do you pick up the threads of an old life',
 'how do you go on when in your heart you begin to understand there is no going back',
 'there are some things that time cannot mend',
 'some hurts that go too deep that have taken hold',
 

In [28]:
separate_sentences = list((mysent.split() for mysent in mydata_tokens_no_punct))
separate_sentences

[['every',
  'day',
  'general',
  'grievous',
  'adds',
  'a',
  'unique',
  'lightsaber',
  'to',
  'his',
  'collection'],
 ['day', '66finale', 'every', 'uraymesiris', 'suggestion'],
 ['uthibson34',
  'has',
  'reddit',
  'premium',
  'for',
  'life',
  'from',
  'these',
  'posts',
  'so',
  'were',
  'all',
  'getting',
  'this',
  'on',
  'a',
  't',
  'shirt',
  'right'],
 ['i',
  'know',
  'one',
  'of',
  'you',
  'quarenteend',
  'at',
  'home',
  'has',
  'a',
  'screen',
  'press'],
 ['lets',
  'get',
  'this',
  'started',
  'my',
  'lightsabers',
  'have',
  'doubled',
  'since',
  'the',
  'last',
  'time',
  'we',
  'met',
  'rprequelmemes',
  'uthibson34',
  'stops',
  'the',
  'series',
  'general',
  'grievous',
  'goodbye',
  'old',
  'friend',
  'may',
  'the',
  'force',
  'be',
  'with',
  'you',
  'and',
  'thus',
  'it',
  'was',
  'the',
  'fourth',
  'age',
  'of',
  'rprequelmemes',
  'began'],
 ['and',
  'the',
  'fellowship',
  'of',
  'grievous',
  'thoug

In [29]:
train, vocab = padded_everygram_pipeline(3, separate_sentences)

In [30]:
lm = MLE(3)

In [31]:
lm.fit(train, vocab)
print(lm.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4047 items>


In [32]:
lm.vocab.lookup(separate_sentences[0])

('every',
 'day',
 'general',
 'grievous',
 'adds',
 'a',
 'unique',
 'lightsaber',
 'to',
 'his',
 'collection')

In [33]:
lm.vocab.lookup(["coffee", "home", "allowed", "call", "america", "trump", "biden", "football", "world"])

('coffee',
 'home',
 '<UNK>',
 'call',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'football',
 'world')

In [34]:
print(lm.counts)

<NgramCounter with 3 ngram orders and 103137 ngrams>


In [35]:
lm.counts['coffee']

1

In [36]:
lm.counts['home']

8

In [37]:
lm.counts['world']

7

In [38]:
lm.generate(3)

['only', '49', 'goodbye']

In [40]:
lm.generate(11)

['<s>',
 '2020',
 'you',
 'just',
 'hold',
 'it',
 'out',
 'in',
 'terror',
 'and',
 'were']

In [41]:
for i in range(2, 10):
    while True:
        f = lm.generate(i)
        if '</s>' not in f and '<s>' not in f:
            break
    print(f)
    

['seen', 'in']
['karma', 'lol', 'fucken']
['rsilverclub', 'i’m', 'at', 'ahhh']
['glance', 'i', 'thought', 'youd', 'make']
['me', 'laugh', 'i', 'cant', 'explain', 'why']
['meme', 'leaves', 'me', 'lost', 'always', 'first', 'pic']
['knew', 'most', 'of', 'his', 'battle', 'bot', 'tournaments', 'with']
['more', 'karma', 'to', 'himself', 'but', 'got', 'banned', 'we', 'miss']


In [42]:
usernames = "200d" #they all contain this code
for i in range(2, 12):
    while True:
        f = lm.generate(i)
        if '</s>' not in f and '<s>' not in f:
            checker = True
            for j in f:
                if usernames in j:
                    checker = False                    
            if checker== True:
                break
    print(f)
    

['sub', 'while']
['proud', 'can', 'we']
['to', 'this', 'post', 'immediately']
['hand', 'now', 'theres', 'just', 'one']
['hand', 'now', 'there', 'are', '65536', 'of']
['ugh', 'why', 'did', 'i', 'hit', 'the', 'all']
['my', 'favorite', 'of', 'his', 'battle', 'bot', 'tournaments', 'with']
['was', 'doing', 'and', 'the', 'schwartz', 'ring', 'as', 'those', 'were']
['day', '63', 'kit', 'fisto’s', 'lightsaber', 'day', '4', 'ahsoka’s', 'lightsaber', 'day']
['the', 'last', 'time', 'we', 'live', 'in', 'i', 'can', 'hear', 'their', 'voices']


In [51]:
usernames = "200d" #they all contain this code
for i in range(2, 12):
    print(f"L = {i}")
    for c in range(3):
        while True:
            f = lm.generate(i)
            if '</s>' not in f and '<s>' not in f:
                checker = True
                for j in f:
                    if usernames in j:
                        checker = False                    
                if checker== True:
                    break
        out=""
        for w in range(len(f)):
            out+=f[w]
            try:
                chegg = f[w+1]
                out+=" "
            except IndexError:
                print(out+".")
        
        
    

L = 2
farmed the.
page i.
years late.
L = 3
live in a.
hear the tragedy.
make this happen.
L = 4
thought youd make one.
a great meme with.
getting out of hand.
L = 5
absolutely devastated 2020 went full.
see you in hot your.
though you did it you.
L = 6
tell me why i love it.
about ani going to tell my.
do the right happened to be.
L = 7
at the top post on reddit absolutely.
f thank you jlha65 for giving 10.
with another 131072 more to get more.
L = 8
testing the myth busters and i dont know.
in this sub reddit for posts like this.
it everyday so i could try to cut.
L = 9
thank you for co hosting one of those watermelon.
comments 1 yoshibacon • 4h how many kyber crystals.
now replace the song wake me up before ya.
L = 10
his comment again with a red lightsaber love how everybody.
the way obi looks at grievous makes it even funnier.
ahaha we’re gonna do it everyday but with a bang.
L = 11
funnier lol uvredditdownloader this is not a story the mods would.
are actually very close cant tel

In [53]:
import random
usernames = "200d" #they all contain this code
for i in range(1, 50):
    r = random.randint(3, 12)
    while True:
        f = lm.generate(r)
        if '</s>' not in f and '<s>' not in f:
            checker = True
            for j in f:
                if usernames in j:
                    checker = False                    
            if checker== True:
                break
    out = " ".join(f)
    out+="."
    print(out, end=" ")

my pretend boyfriend for those of you. is the highest upvoted post on this planet but we. for the cough. actually be funny i love. unfold well theres only. this in my life my creme. all the lightsabers igniting generalgbt grevious i like your thing. amount of coins spent on this planet but we can all. may the force be with you day 40. 2ewan2mcgregor coruscant drift 6 mandaloredashyousa. cybertechlabs • 4h. missed rejoice for those who come after me i thought. love the b1. to be on tv helped to fuel my passion for. is why i’m happy that we. needed this video this is a. to be left. my upvote for taking time into making quality drawings will do. trained to detect. this into a dive. is the art begins. armed and operational battle station. upgrades attack kenobi incredible. screen in these movies. when it reaches the front page or something i. wars and game of thrones art for the hand in the background. ate my cookie. shitposters yourintensifies clonesintensifies areintensifies veryintensif

##### experimenting with n gram paramater

In [55]:
usernames = "200d" #they all contain this code

for N in range(2, 6):

    trainx, vocabx = padded_everygram_pipeline(N, separate_sentences)
    lmx = MLE(N)
    lmx.fit(trainx, vocabx)
    
    print(f"\n{N}-gram trained sentences")

    for i in range(1, 21):
        r = random.randint(N, 12)
        while True:
            f = lmx.generate(r)
            if '</s>' not in f and '<s>' not in f:
                checker = True
                for j in f:
                    if usernames in j:
                        checker = False                    
                if checker== True:
                    break
        out = " ".join(f)
        out+="."
        print(i, ":  ", out, end="\n")


2-gram trained sentences
1 :   the most upvoted post on my childhood on this became.
2 :   that could see through.
3 :   i thought i think so.
4 :   pure fucking awesome i say hello there congrats on.
5 :   we get any money id love this upvote him to.
6 :   a chance.
7 :   rice one of comic with quigons face i promise ill upbote that.
8 :   when filming episode 6.
9 :   this is better holy craprip man rip grant was somehow hurts i.
10 :   the thing the art.
11 :   had a bunch of him too.
12 :   grant for the fuck magnificent.
13 :   be balanced ahh the mythbusters on reddit should be.
14 :   you into physics principles behind geez on.
15 :   lol how many people find.
16 :   dude look if the force will always 131072 of them.
17 :   rip mr sulu seemed like what are.
18 :   end rip i like that” i wanna buy some shameless.
19 :   are strong and old.
20 :   edit i.

3-gram trained sentences
1 :   one helloooooo there.
2 :   what he must.
3 :   3h aim right above the top post of this subred

In [61]:
usernames = "200d" #they all contain this code

for N in range(2, 6):

    trainx, vocabx = padded_everygram_pipeline(N, separate_sentences)
    lmx = MLE(N)
    lmx.fit(trainx, vocabx)
    
    print(f"\n{N}-gram trained sentences")

    for i in range(1, 21):
        r = random.randint(N, 12)
        while True:
            f = lmx.generate(r)
            if '</s>' not in f and '<s>' not in f:
                checker = True
                for j in f:
                    if usernames in j:
                        checker = False                    
                if checker== True:
                    break
        out = " ".join(f)
        out+="."
        print(i, ":  ", out, end="\n")


2-gram trained sentences
1 :   barriss offee’s lightsaber day 21 luke’s food yoda ate.
2 :   and someone is the most upvoted post on chromehttpschromegooglecomwebstoredetailmentionedvideosforreddfiimkmdalmgffhibfdjnhljpnigcmohf.
3 :   the chance to the oc.
4 :   amazing deleted i get me of planet but.
5 :   to continue to.
6 :   bunch of control i dont need a bold.
7 :   no this.
8 :   again on the previous grievous experience which one younger and upload it.
9 :   light saber emerge like a bad feeling.
10 :   this picture of the.
11 :   he tried to here doing this is too much i can’t do.
12 :   on every award for discerning.
13 :   a trick i the upvotes please draw mustar obi wans are getting.
14 :   for when he got us too deep there this is god.
15 :   grievous goodbye old posts of the most upvoted.
16 :   “jedi master kenobi how much rise of naboo 7 obi wan.
17 :   this close.
18 :   really appreciated this needs to.
19 :   and every day 10 points gryffindor 18270 hufflepuff 16820 

### Part 3

In [62]:
subr_no_punct

['every',
 'day',
 'general',
 'grievous',
 'adds',
 'a',
 'unique',
 'lightsaber',
 'to',
 'his',
 'collection',
 'day',
 '66',
 'finale',
 'every',
 'suggestion',
 'has',
 'reddit',
 'premium',
 'for',
 'life',
 'from',
 'these',
 'posts',
 'so',
 'were',
 'all',
 'getting',
 'this',
 'on',
 'a',
 't',
 'shirt',
 'right',
 'i',
 'know',
 'one',
 'of',
 'you',
 'quarenteend',
 'at',
 'home',
 'has',
 'a',
 'screen',
 'press',
 'let',
 'get',
 'this',
 'started',
 'my',
 'lightsabers',
 'have',
 'doubled',
 'since',
 'the',
 'last',
 'time',
 'we',
 'met',
 'stops',
 'the',
 'series',
 'general',
 'grievous',
 'goodbye',
 'old',
 'friend',
 'may',
 'the',
 'force',
 'be',
 'with',
 'you',
 'and',
 'thus',
 'it',
 'was',
 'the',
 'fourth',
 'age',
 'of',
 'began',
 'and',
 'the',
 'fellowship',
 'of',
 'grievous',
 'though',
 'eternally',
 'bound',
 'by',
 'friendship',
 'and',
 'love',
 'was',
 'ended',
 'nine',
 'and',
 'a',
 'half',
 'weeks',
 'to',
 'the',
 'day',
 'since',
 'thibso

In [63]:
import numpy as np

In [64]:
sub_nump = np.array(subr_no_punct)

In [65]:
sub_nump.flatten()

array(['every', 'day', 'general', ..., 'my', 'internet', 'for'],
      dtype='<U145')

In [66]:
sub_nump[:20]

array(['every', 'day', 'general', 'grievous', 'adds', 'a', 'unique',
       'lightsaber', 'to', 'his', 'collection', 'day', '66', 'finale',
       'every', 'suggestion', 'has', 'reddit', 'premium', 'for'],
      dtype='<U145')

In [67]:
sub_nump = np.array([list(i) for i in sub_nump])
 

C:\Users\User\AppData\Local\Temp\ipykernel_20528\3295353971.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sub_nump = np.array([list(i) for i in sub_nump])


In [68]:
sub_nump

array([list(['e', 'v', 'e', 'r', 'y']), list(['d', 'a', 'y']),
       list(['g', 'e', 'n', 'e', 'r', 'a', 'l']), ..., list(['m', 'y']),
       list(['i', 'n', 't', 'e', 'r', 'n', 'e', 't']),
       list(['f', 'o', 'r'])], dtype=object)

In [69]:
# subr_tokens 
train2, vocab2 = padded_everygram_pipeline(3, sub_nump)
wlm = MLE(3)


In [70]:
wlm.fit(train2, vocab2)
print(wlm.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 50 items>


In [71]:
wlm.vocab.lookup(subr_tokens[0])

'<UNK>'

In [72]:
for i in range(1, 50):
    r = random.randint(3, 12)
    while True:
        f = wlm.generate(r)
        if '</s>' not in f and '<s>' not in f:
            checker = True
            for j in f:
                if usernames in j:
                    checker = False                    
            if checker== True:
                break
    out = "".join(f)
    out+=" "
    print(out, end=" ")

edit  notsabeeee  irom  erazinkillo  arne  youslovere  wanksguysmo  sellsentin  pickinit  lefteri  thentestact  faleftel  rommemprevot  abeig  sterve  atectupv  memplarm  gnutin  upvouturpril  icirst  evensiblarth  isay  ibloo  eents  thesternster  sad  gens  vereacti  ymorgionstut  thiblightto  end  petactifuld  ery  tbron  tinget  ible  rie  dearandkil  lectifuc  lightsaboa  poinaribligh  anch  oureaddit  anythounfing  cand  rontsandea  fromm  haverievede  thosecound  

In [73]:
usernames = "200d" #they all contain this code

for N in range(2, 8):

    train2x, vocab2x = padded_everygram_pipeline(N, sub_nump)
    wlmx = MLE(N)

    wlmx.fit(train2x, vocab2x)
    
    print(f"\n{N}-gram trained words")

    for i in range(1, 21):
        r = random.randint(2, 12)
        while True:
            f = wlmx.generate(r)
            if '</s>' not in f and '<s>' not in f:
                checker = True
                for j in f:
                    if usernames in j:
                        checker = False                    
                if checker== True:
                    break
        out = "".join(f)
        out+=" "
        print(out, end=" ")


2-gram trained words
is  amannisan  ousastangist  augrrve  onyoud  usuddayw  pou  blethi  outisinghith  sonthoung  surnthec  chinerer  uthely  cofung  elll  arthar  nt  iofudika  bouare  ofuer  
3-gram trained words
anatior  alnalkell  ecturefors  eackgri  rou  averfuch  trawante  herio  ovion  ntifeepi  athe  ghtsavour  likeen  mememes  ncruleati  an  egainge  eseeenterac  otticer  anceirse  
4-gram trained words
me  pproughtsabe  greviou  eaths  powerfectat  link  pperm  generator  icondow  stageous  his  lculou  verybo  ou  hing  aaaaaaaa  wan2mcgrew  mallightsab  imaginsand  abreall  
5-gram trained words
aroun  rid  be  splas  ght  thin  ntertainmen  opportantles  eeeeeeeee  fication  mythbusterpi  emationised  footage  reque  eeeeeeeeeeee  ul  all  multimation  cough  les  
6-gram trained words
herfucking  beautifu  thibbbbbbbss  ed  comments  track2onsta  erybody  athemati  ccordinate  droidificat  autifu  achievemen  eeeeeeeeeee  permabanne  expres  tmarsupial  disturb  favour

In [77]:
from nltk.corpus import wordnet

for N in range(2, 10):

    train2x, vocab2x = padded_everygram_pipeline(N, sub_nump)
    wlmx = MLE(N)

    wlmx.fit(train2x, vocab2x)
    
    print(f"\n{N}-gram real words produced percentage")
    
    wordarr=[]

    for i in range(0, 1000):
        r = random.randint(2, 12)
        while True:
            f = wlmx.generate(r)
            if '</s>' not in f and '<s>' not in f:
                checker = True
                for j in f:
                    if usernames in j:
                        checker = False                    
                if checker== True:
                    break
        out = "".join(f)
        wordarr.append(out)
    count=0
    for w in wordarr:
        if wordnet.synsets(w):
            count+=1
    perc = count/10
    print(perc)
          


2-gram real words produced percentage
11.2

3-gram real words produced percentage
11.5

4-gram real words produced percentage
15.1

5-gram real words produced percentage
22.8

6-gram real words produced percentage
22.0

7-gram real words produced percentage
26.4


In [78]:
from nltk.corpus import wordnet

for N in range(2, 10):

    train2x, vocab2x = padded_everygram_pipeline(N, sub_nump)
    wlmx = MLE(N)

    wlmx.fit(train2x, vocab2x)
    
    print(f"\n{N}-gram real words produced percentage")
    
    wordarr=[]

    for i in range(0, 1000):
        r = random.randint(2, 12)
        while True:
            f = wlmx.generate(r)
            if '</s>' not in f and '<s>' not in f:
                checker = True
                for j in f:
                    if usernames in j:
                        checker = False                    
                if checker== True:
                    break
        out = "".join(f)
        wordarr.append(out)
    count=0
    for w in wordarr:
        if wordnet.synsets(w):
            count+=1
    perc = count/10
    print(perc)
          


2-gram real words produced percentage
9.5

3-gram real words produced percentage
15.2

4-gram real words produced percentage
17.8

5-gram real words produced percentage
20.8

6-gram real words produced percentage
24.4

7-gram real words produced percentage
24.1

8-gram real words produced percentage
24.8

9-gram real words produced percentage
24.7
